In [1]:
import sys
sys.path.append('..')

In [2]:

%load_ext snakeviz

In [3]:
from utils_reboot.datasets import Dataset

dataset = Dataset("wine", path = "../data/real/")
dataset.drop_duplicates()

In [4]:
from model_reboot.EIF_reboot import ExtendedTree, ExtendedIsolationForest
import numpy as np

In [5]:
I=ExtendedIsolationForest(True)

In [6]:
I.fit(dataset.X)

In [7]:
# from pyinstrument import Profiler
# profiler = Profiler()
# profiler.start()
# I.fit(dataset.X)
# profiler.stop()
# profiler.print()

In [8]:
dataset.y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
m = np.c_[I.predict(dataset.X),dataset.y]
m[np.argsort(m[:,0,])][::-1]

<string>:3: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))


array([[0.56862193, 1.        ],
       [0.56166942, 1.        ],
       [0.56112128, 0.        ],
       [0.55545233, 1.        ],
       [0.55534387, 0.        ],
       [0.55512702, 0.        ],
       [0.54994802, 0.        ],
       [0.54951863, 1.        ],
       [0.54855371, 1.        ],
       [0.54833951, 0.        ],
       [0.54759049, 1.        ],
       [0.54598888, 0.        ],
       [0.54598888, 1.        ],
       [0.54545605, 0.        ],
       [0.54534954, 0.        ],
       [0.54524306, 0.        ],
       [0.54471096, 0.        ],
       [0.54332992, 0.        ],
       [0.54290569, 0.        ],
       [0.54279969, 0.        ],
       [0.5426937 , 0.        ],
       [0.54258774, 1.        ],
       [0.54195239, 0.        ],
       [0.54184657, 0.        ],
       [0.5414235 , 0.        ],
       [0.54110641, 0.        ],
       [0.54100076, 0.        ],
       [0.54057835, 0.        ],
       [0.54057835, 1.        ],
       [0.54036727, 0.        ],
       [0.

# Test time

In [10]:

%snakeviz --new-tab I.fit(dataset.X)

 
*** Profile stats marshalled to file '/var/folders/m5/gwy76h051b78cmrncgncqv880000gn/T/tmpilo3d7f5'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fvar%2Ffolders%2Fm5%2Fgwy76h051b78cmrncgncqv880000gn%2FT%2Ftmpilo3d7f5


In [12]:
import tqdm
dataset = Dataset("wine", path = "../data/real/")
dataset.drop_duplicates()
I=ExtendedIsolationForest(True,n_estimators=400)
for _ in tqdm.tqdm(range(10)):
    I.fit(dataset.X)
    I.predict(dataset.X)


 10%|█         | 1/10 [00:15<02:22, 15.85s/it]


KeyboardInterrupt: 